In [64]:
import pandas as pd

In [65]:
# no of questions marked,
# no of questions soft correct (0.5 for Y-S or N-S, 1 for Y-N) and hard correct, 

# no of easy questions marked,
# no of questions soft correct (0.5 for Y-S or N-S, 1 for Y-N) and hard correct,
# No of snippets for easy questions correctly marked

In [66]:
num_ques = 5

df = pd.read_csv('data/all_1000_batch1_final_answers.csv')

easy_df = pd.read_csv('data/easy_data.csv')
easy_df = easy_df[["id", "is_answerable", "relevant_snippets"]]
easy_df.is_answerable.replace("Y", 1, inplace=True)
easy_df.is_answerable.replace("S", 2, inplace=True)
easy_df.is_answerable.replace("N", 3, inplace=True)
easy_df["easy_ans"] = easy_df["is_answerable"]
easy_df["easy_snips"] = easy_df["relevant_snippets"]
easy_df.drop(columns=["is_answerable", "relevant_snippets"], inplace=True)

df["uid"] = ""
df["uid"] = df.uid.str.cat([df["Input.id" + str(i+1)] for i in range(num_ques)], '#')


df["easy_result"] = ""
df["easy_id"] = ""
for hit in range(len(df)):
    for i in range(num_ques):
        id_ = df["Input.id" + str(i + 1)].iloc[hit]
        if id_.startswith('('):    
            df.at[hit, "easy_id"] = id_
            df.at[hit, "easy_result"] = df.iloc[hit]["Answer.Q" + str(i + 1) + "ans"]
            break

        
df.set_index("uid", inplace=True)

print(len(df))
df = pd.merge(df, easy_df, how='inner', left_on='easy_id', right_on='id')
print(len(df))

print(df.columns)

250
223
Index(['HITId', 'HITTypeId', 'Title', 'Description', 'Keywords', 'Reward',
       'CreationTime', 'MaxAssignments', 'RequesterAnnotation',
       'AssignmentDurationInSeconds', 'AutoApprovalDelayInSeconds',
       'Expiration', 'NumberOfSimilarHITs', 'LifetimeInSeconds',
       'AssignmentId', 'WorkerId', 'AssignmentStatus', 'AcceptTime',
       'SubmitTime', 'AutoApprovalTime', 'ApprovalTime', 'RejectionTime',
       'RequesterFeedback', 'WorkTimeInSeconds', 'LifetimeApprovalRate',
       'Last30DaysApprovalRate', 'Last7DaysApprovalRate', 'Input.id1',
       'Input.question1', 'Input.Snippet11', 'Input.Snippet12',
       'Input.Snippet13', 'Input.Snippet14', 'Input.Snippet15', 'Input.id2',
       'Input.question2', 'Input.Snippet21', 'Input.Snippet22',
       'Input.Snippet23', 'Input.Snippet24', 'Input.Snippet25', 'Input.id3',
       'Input.question3', 'Input.Snippet31', 'Input.Snippet32',
       'Input.Snippet33', 'Input.Snippet34', 'Input.Snippet35', 'Input.id4',
       'In

In [67]:
df['easy_id']

0       (576,2)
1       (576,2)
2       (576,2)
3       (576,2)
4       (576,2)
5       (576,2)
6       (576,2)
7       (576,2)
8       (576,2)
9       (576,2)
10      (576,2)
11      (576,2)
12      (576,2)
13      (576,2)
14      (576,2)
15      (576,2)
16      (576,2)
17      (576,2)
18      (576,2)
19      (576,2)
20      (576,2)
21      (576,2)
22      (576,2)
23      (576,2)
24      (576,2)
25      (576,2)
26      (576,2)
27      (576,2)
28     (2184,4)
29     (2184,4)
         ...   
193    (3956,0)
194    (3956,0)
195    (3956,0)
196    (4516,4)
197    (4516,4)
198    (4516,4)
199    (4516,4)
200    (4516,4)
201    (4516,4)
202    (4516,4)
203    (4516,4)
204    (4516,4)
205    (4516,4)
206    (4516,4)
207    (4516,4)
208    (4516,4)
209    (4516,4)
210    (4516,4)
211    (4516,4)
212    (4516,4)
213    (4516,4)
214    (4516,4)
215    (4516,4)
216    (4516,4)
217    (4516,4)
218    (4516,4)
219    (4516,4)
220    (4516,4)
221    (4516,4)
222    (4516,4)
Name: easy_id, Length: 2

In [68]:
df["easy_result_SY"] = df["easy_result"].replace(2, 1)#Y, S, N
df["easy_result_SY"] = df["easy_result_SY"].replace(3, 2)

df["easy_result_NY"] = df["easy_result"].replace(3, 2)

df["easy_ans_SY"] = df["easy_ans"].replace(2, 1)
df["easy_ans_SY"] = df["easy_ans_SY"].replace(3, 2)

df["easy_ans_NY"] = df["easy_ans"].replace(3, 2)

In [69]:
L = len(df)

print( sum(df.easy_result == df.easy_ans ) / L * 1.0)
print( sum(df.easy_result_SY == df.easy_ans_SY ) / L * 1.0)
print( sum(df.easy_result_NY == df.easy_ans_NY ) / L * 1.0)

print(L)

0.39461883408071746
0.5515695067264574
0.726457399103139
223


In [70]:
sdf = df[["easy_ans_SY", "easy_result_SY", "easy_ans_NY", "easy_result_NY"]].dropna()
from sklearn.metrics import classification_report
print(classification_report(sdf.easy_ans_SY, sdf.easy_result_SY))
print(classification_report(sdf.easy_ans_NY, sdf.easy_result_NY))
#len(sdf), len(sdf.easy_result)

             precision    recall  f1-score   support

          1       0.53      0.90      0.67       112
          2       0.67      0.20      0.31       111

avg / total       0.60      0.55      0.49       223

             precision    recall  f1-score   support

          1       0.46      0.55      0.50        56
          2       0.84      0.78      0.81       167

avg / total       0.75      0.73      0.73       223



In [71]:
df["easy_sum_SY"] = df.easy_result_SY + df.easy_ans_SY 

print( sum(df["easy_sum_SY"] == 2) / sum(df.easy_result_SY == 1) )
print( sum(df["easy_sum_SY"] == 2) / sum(df.easy_ans_SY == 1) )

print(sum(df["easy_sum_SY"] == 2), sum(df.easy_ans_SY == 1), sum(df.easy_result_SY == 1))

0.531578947368421
0.9017857142857143
101 112 190


In [8]:
df["easy_sum_NY"] = df.easy_result_NY + df.easy_ans_NY 

print( sum(df["easy_sum_NY"] == 2) / sum(df.easy_result_NY == 1) )
print( sum(df["easy_sum_NY"] == 2) / sum(df.easy_ans_NY == 1) )

print(sum(df["easy_sum_NY"] == 2), sum(df.easy_ans_NY == 1), sum(df.easy_result_NY == 1))

0.43
0.7747747747747747
86 111 200


In [9]:
sum(df["easy_ans_SY"] == 1) / len(df)*1.

0.5171673819742489

In [10]:
len(df)

466

In [11]:
print(len(df[df.easy_result == 1]))
print(len(df[df.easy_result == 2]))
print(len(df[df.easy_result == 3]))

200
129
133


In [12]:
print(len(easy_df[easy_df.easy_ans == 1]))
print(len(easy_df[easy_df.easy_ans == 2]))
print(len(easy_df[easy_df.easy_ans == 3]))

2
2
4


In [13]:
easy_df

,id,easy_ans,easy_snips
0,"(3498,4)",3,NaN
1,"(3971,6)",2,2
2,"(576,2)",3,NaN
3,"(2184,4)",3,NaN
4,"(3131,0)",2,1
5,"(1439,5)",1,1|2|3|4|5
6,"(3956,0)",1,1|2
7,"(4516,4)",3,NaN


In [14]:
small_df = df[["Input.question1","Input.question2","Input.question3","Input.question4","Input.question5", "easy_ans", "easy_result"]]

In [15]:
print(len(df[df.easy_ans == 1]))#Y
print(len(df[df.easy_ans == 2]))#S
print(len(df[df.easy_ans == 3]))#N

111
130
225


In [16]:
print(len(df[df.easy_result == 1]))
print(len(df[df.easy_result == 2]))
print(len(df[df.easy_result == 3]))

200
129
133


In [17]:
df["easy_correct_SY"] = df["easy_ans_SY"] == df["easy_result_SY"]
len(df)

466

In [18]:
arr = []
for hit in range(len(df)):
    arr.append((df.iloc[hit]["WorkerId"], df.iloc[hit]["easy_correct_SY"]))

sorted(arr)

[('A10K5SME6K37U2', True),
 ('A10Z5S5QDNMA9O', False),
 ('A10Z5S5QDNMA9O', False),
 ('A1127U06EX2U6H', True),
 ('A11R9XFIQDY9SP', False),
 ('A12KINXC5RV6OV', True),
 ('A13GUTDMGGZZDL', False),
 ('A13GUTDMGGZZDL', False),
 ('A13GUTDMGGZZDL', False),
 ('A13GUTDMGGZZDL', False),
 ('A13GUTDMGGZZDL', True),
 ('A13GUTDMGGZZDL', True),
 ('A13GUTDMGGZZDL', True),
 ('A13GUTDMGGZZDL', True),
 ('A13GUTDMGGZZDL', True),
 ('A13GUTDMGGZZDL', True),
 ('A13GUTDMGGZZDL', True),
 ('A13GUTDMGGZZDL', True),
 ('A13GUTDMGGZZDL', True),
 ('A174VKTMI6268M', False),
 ('A17E4A0FBN9RCW', False),
 ('A17E4A0FBN9RCW', False),
 ('A17E4A0FBN9RCW', True),
 ('A17E4A0FBN9RCW', True),
 ('A17E4A0FBN9RCW', True),
 ('A17E4A0FBN9RCW', True),
 ('A17E4A0FBN9RCW', True),
 ('A17T285YLOZKZQ', False),
 ('A17T285YLOZKZQ', False),
 ('A17T285YLOZKZQ', False),
 ('A17T285YLOZKZQ', False),
 ('A17T285YLOZKZQ', True),
 ('A17T285YLOZKZQ', True),
 ('A1958FL5DWE5SQ', False),
 ('A1958FL5DWE5SQ', False),
 ('A19LKLAPLXL9B1', False),
 ('A1A0JZNM

In [19]:
df[["WorkerId","easy_result_SY"]]

,WorkerId,easy_result_SY
0,A2W23ILRCA9VD1,1.0
1,A1QUOTZEN3BNVF,1.0
2,A2UCRP35ZYHPJ3,1.0
3,A3HCEK0YKU186K,1.0
4,A3BTK49X4ITNXJ,1.0
5,A3HCEK0YKU186K,1.0
6,A2XC9IWFQ3FWUR,1.0
7,A3W5BTNHDA9U64,1.0
8,A3696JXTRKL2FI,1.0
9,ALHAO8VJDTR7G,1.0


In [20]:
len(df)

466

In [21]:
accuracy_per_worker = df.groupby(['WorkerId'])["easy_correct_SY"].mean().to_frame()
accuracy_per_worker.rename(index=str, columns={"easy_correct_SY": "worker_accuracy_SY"}, inplace=True)
accuracy_per_worker['WorkerId'] = accuracy_per_worker.index

In [63]:
df.groupby(['WorkerId'])["easy_correct_SY"].count().max()

In [22]:
len(accuracy_per_worker)

113

In [23]:
df = pd.merge(df, accuracy_per_worker, how='outer', left_on='WorkerId', right_on='WorkerId')

In [24]:
len(df)

466

In [25]:
approved = 0
rejected = 0

def accept_func(x):
    global approved
    if x > 0.6:
        approved += 1
        return "x"
    return ''

def reject_func(x):
    global rejected
    if x <= 0.6:
        rejected += 1
        return "Worker's overall accuracy very low"
    return ''

df.Approve = df.worker_accuracy_SY.apply(accept_func)
df.Reject = df.worker_accuracy_SY.apply(reject_func)

In [26]:
df[["worker_accuracy_SY", "Approve"]]

,worker_accuracy_SY,Approve
0,1.000000,x
1,0.666667,x
2,0.666667,x
3,0.666667,x
4,0.666667,x
5,0.666667,x
6,0.666667,x
7,1.000000,x
8,1.000000,x
9,0.600000,


In [27]:
approved

242

In [28]:
rejected

224

In [29]:
original_columns = pd.read_csv('data/all_1000_batch1_final_answers.csv').columns

In [30]:
out_df = df[original_columns]

In [31]:
out_df.to_csv('all_1000_batch1_final_answers_approve_reject.csv', index=False)

In [32]:
mv all_1000_batch1_final_answers_approve_reject.csv data/all_1000_batch1_final_answers_approve_reject.csv

In [33]:
ls data

all_100000.csv
all_100000_final.csv
all_1000_batch1_final.csv
all_1000_batch1_final_answers.csv
all_1000_batch1_final_answers_approve_reject.csv
all_2000_batch1_final_answers.csv
all_2000_batch1_upload.csv
all_2000_batch1_upload_utf8.csv
all_2000_batch1_upload_utf8_final.csv
easy_data.csv
electronics_10000.csv
electronics_10000_final.csv
original_easy_data.csv


In [36]:
accuracy_per_worker.sort_values(by="worker_accuracy_SY")

,worker_accuracy_SY,WorkerId
WorkerId,,
APHNYDGTCRN3O,0.000000,APHNYDGTCRN3O
A2R6IR3Z2CC71R,0.000000,A2R6IR3Z2CC71R
AOH6EYODJT6V7,0.000000,AOH6EYODJT6V7
A2NGY0LAM66P0N,0.000000,A2NGY0LAM66P0N
A5024WK5CE1BT,0.000000,A5024WK5CE1BT
A2331BYL6VEM6X,0.000000,A2331BYL6VEM6X
A24JKHC4HTY6CD,0.000000,A24JKHC4HTY6CD
A34WUGMV8YJGFS,0.000000,A34WUGMV8YJGFS
ASCLZFXD9WMON,0.000000,ASCLZFXD9WMON


In [38]:
df.easy_result.value_counts()

1.0    200
3.0    133
2.0    129
Name: easy_result, dtype: int64

In [39]:
len(df)

466

In [41]:
df.easy_result.describe()

count     462.0
unique      3.0
top         1.0
freq      200.0
Name: easy_result, dtype: float64

In [44]:
df[["easy_ans", "easy_result"]]

,easy_ans,easy_result
0,1,1
1,1,1
2,3,3
3,3,3
4,3,3
5,2,3
6,3,1
7,1,1
8,1,1
9,1,1
